<a href="https://colab.research.google.com/github/Username1234jj/AI-Research-Assistant/blob/main/AI_Research_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Reasearch Assistant - **Setup**

In [ ]:
# 🧠 AI Research Assistant - Setup

!pip install PyMuPDF transformers gradio torch torchvision torchaudio --quiet

import fitz  # PyMuPDF
from transformers import pipeline
import gradio as gr

print("✅ All libraries installed and imported successfully!")


# Upload and Extract Text from **PDF**

In [ ]:
# 🧾 Step 2 - Upload and Extract Text from PDF
from google.colab import files
import fitz  # PyMuPDF

print("📤 Please upload your research paper (PDF)...")
uploaded = files.upload()

pdf_text = ""
for filename in uploaded.keys():
    with fitz.open(filename) as pdf:
        for page in pdf:
            pdf_text += page.get_text()

print(f"✅ Extracted {len(pdf_text)} characters from {list(uploaded.keys())[0]}")


# Summarize the Research Paper using AI

In [ ]:
# 🤖 Step 3 - Summarize the Research Paper Text
!pip install transformers --quiet

from transformers import pipeline

# Load a summarization model (free + works on Colab CPU)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Split text into manageable chunks
max_chunk = 1000
chunks = [pdf_text[i:i+max_chunk] for i in range(0, len(pdf_text), max_chunk)]

summary = ""
for i, chunk in enumerate(chunks):
    print(f"🧩 Summarizing chunk {i+1}/{len(chunks)} ...")
    out = summarizer(chunk, max_length=200, min_length=50, do_sample=False)
    summary += out[0]['summary_text'] + " "

print("\n✅ Summary Generated Successfully!\n")
print(summary[:3000])  # Print first part of summary


# Ask Questions About the Paper (Q&A System)

In [ ]:
# 💬 Step 4 - Ask Questions About the Paper
!pip install transformers --quiet

from transformers import pipeline

# Load a question-answering model
qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")

print("🤖 Your AI Research Assistant is ready!")
print("You can now ask questions about the paper.\n")

while True:
    question = input("❓ Ask a question (or type 'exit' to stop): ")
    if question.lower() == "exit":
        print("👋 Session ended. Great job, researcher!")
        break
    result = qa_model(question=question, context=pdf_text)
    print(f"💡 Answer: {result['answer']}\n")


# **Create a Web App Interface with Gradio**

In [ ]:
# 🌐 Step 5 - Web App for the AI Research Assistant
!pip install gradio --quiet
import gradio as gr
from transformers import pipeline

# Load models
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")

def summarize_paper(pdf_text):
    max_chunk = 1000
    chunks = [pdf_text[i:i+max_chunk] for i in range(0, len(pdf_text), max_chunk)]
    summary = ""
    for chunk in chunks:
        out = summarizer(chunk, max_length=200, min_length=50, do_sample=False)
        summary += out[0]['summary_text'] + " "
    return summary

def answer_question(question, pdf_text):
    if not question.strip():
        return "Please type a question."
    result = qa_model(question=question, context=pdf_text)
    return result['answer']

# Define Gradio interface
with gr.Blocks() as ai_research_assistant:
    gr.Markdown("# 🤖 AI Research Assistant")
    gr.Markdown("Upload a research paper, summarize it, and ask questions!")

    pdf_input = gr.File(label="Upload your research paper (PDF)")
    summary_output = gr.Textbox(label="Summary", lines=10)
    question_input = gr.Textbox(label="Ask a question about the paper")
    answer_output = gr.Textbox(label="Answer")

    summarize_btn = gr.Button("Summarize Paper")
    ask_btn = gr.Button("Ask Question")

    pdf_state = gr.State("")

    def extract_text_from_pdf(pdf_file):
        import fitz
        text = ""
        with fitz.open(pdf_file.name) as pdf:
            for page in pdf:
                text += page.get_text()
        return text

    summarize_btn.click(fn=lambda f: summarize_paper(extract_text_from_pdf(f)), inputs=pdf_input, outputs=summary_output)
    ask_btn.click(fn=lambda q, f: answer_question(q, extract_text_from_pdf(f)), inputs=[question_input, pdf_input], outputs=answer_output)

ai_research_assistant.launch()
